In [ ]:
#pip install -U langchain langchain-community langchain-openai faiss-cpu pypdf

## Load the PDF

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

PDF_PATH = "data/Introduction to Neural Networks.pdf"
loader = PyPDFLoader(PDF_PATH)
docs = loader.load()  # one Document per page


In [4]:
docs

[Document(metadata={'producer': 'PyPDF', 'creator': 'Google', 'creationdate': '', 'title': 'Introduction to Neural Networks', 'source': 'data/Introduction to Neural Networks.pdf', 'total_pages': 83, 'page': 0, 'page_label': '1'}, page_content='Introduction to Neural \nNetworks'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'Google', 'creationdate': '', 'title': 'Introduction to Neural Networks', 'source': 'data/Introduction to Neural Networks.pdf', 'total_pages': 83, 'page': 1, 'page_label': '2'}, page_content='Agenda - Schedule\n1. Historical Intro to Neural Nets\n2. Perceptrons\n3. Break\n4. Gradient Descent\n5. Simple Feed Forward Networks Hyperplanes, bayes theorem, neighbors, why \nnot just take inspiration from the brain?'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'Google', 'creationdate': '', 'title': 'Introduction to Neural Networks', 'source': 'data/Introduction to Neural Networks.pdf', 'total_pages': 83, 'page': 2, 'page_label': '3'}, page_content='Agenda 

## Split into chunks

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)
print("Chunks:", len(chunks))

Chunks: 117


In [7]:
chunks

[Document(metadata={'producer': 'PyPDF', 'creator': 'Google', 'creationdate': '', 'title': 'Introduction to Neural Networks', 'source': 'data/Introduction to Neural Networks.pdf', 'total_pages': 83, 'page': 0, 'page_label': '1'}, page_content='Introduction to Neural \nNetworks'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'Google', 'creationdate': '', 'title': 'Introduction to Neural Networks', 'source': 'data/Introduction to Neural Networks.pdf', 'total_pages': 83, 'page': 1, 'page_label': '2'}, page_content='Agenda - Schedule\n1. Historical Intro to Neural Nets\n2. Perceptrons\n3. Break\n4. Gradient Descent\n5. Simple Feed Forward Networks Hyperplanes, bayes theorem, neighbors, why \nnot just take inspiration from the brain?'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'Google', 'creationdate': '', 'title': 'Introduction to Neural Networks', 'source': 'data/Introduction to Neural Networks.pdf', 'total_pages': 83, 'page': 2, 'page_label': '3'}, page_content='Agenda 

## Embed & store

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(api_key="")  
db = FAISS.from_documents(chunks, embeddings)
retriever = db.as_retriever()

## Build Q&A chain

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key="",model="gpt-4o-mini", temperature=0)
qa = RetrievalQA.from_chain_type(llm, retriever=retriever)

query = "What is the main conclusion of the PDF?"
print(qa.run(query))

/var/folders/q5/2q5fx7g95dz8n7ybspx1lw8r0000gp/T/ipykernel_8752/506504265.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(qa.run(query))


I don't know.


In [12]:
query = "what is a neuron?"
print(qa.run(query))

A neuron is a cell that takes input and provides output via electrical signals, known as action potentials. It "fires" and sends a signal to another neuron when it reaches a certain level of depolarization, typically around -55 microvolts. Neurons are fundamental components of the central and peripheral nervous systems.


In [13]:
print(qa.run('What is z?'))

I don't know.


In [14]:
qa.run('Can you show me a deep neural network diagram?')

"I don't know."

____________________________________________________